<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보
    
</div>

In [79]:
import pandas as pd
path = './olist_ecommerce_data/'
products = pd.read_csv( path + 'olist_products_dataset.csv', encoding = 'utf-8-sig')
customers = pd.read_csv( path + 'olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocation = pd.read_csv( path + 'olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
order_items = pd.read_csv( path + 'olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv( path + 'olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv( path + 'olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv( path + 'olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv( path + 'olist_sellers_dataset.csv', encoding = 'utf-8-sig')
category_name = pd.read_csv( path + 'product_category_name_translation.csv', encoding = 'utf-8-sig')

### [ 1. 월별 평균 거래액 분석 ]

#### (1) 데이터 분석 전 해야할 일
- Data cleansing or data cleaning
- 가장 기본적인 것은 없는 데이터를 확인하는 일

In [80]:
# 분석을 위해 order_purchase_timestamp 사용
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [81]:
# 분석을 위해 payment_value 사용
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


- NaN 데이터 확인하기

In [82]:
orders.info() #  NaN 데이터 확인됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [83]:
payments.info() # NaN 데이터 X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [84]:
# order_approved_at, order_delivered_carrier_date, order_delivered_customer_date에서 NaN 확인
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [85]:
# dropna를 이용해 데이터가 없는 행은 삭제시킨다.
orders = orders.dropna()
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96461 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96461 non-null  object
 1   customer_id                    96461 non-null  object
 2   order_status                   96461 non-null  object
 3   order_purchase_timestamp       96461 non-null  object
 4   order_approved_at              96461 non-null  object
 5   order_delivered_carrier_date   96461 non-null  object
 6   order_delivered_customer_date  96461 non-null  object
 7   order_estimated_delivery_date  96461 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


#### (2) orders 와 payments 매칭전 사이즈 확인

In [86]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96461 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96461 non-null  object
 1   customer_id                    96461 non-null  object
 2   order_status                   96461 non-null  object
 3   order_purchase_timestamp       96461 non-null  object
 4   order_approved_at              96461 non-null  object
 5   order_delivered_carrier_date   96461 non-null  object
 6   order_delivered_customer_date  96461 non-null  object
 7   order_estimated_delivery_date  96461 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


In [87]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


- 사이즈 크기가 서로다르기 떄문에, 매칭에 어려움이 보인다. 따라서, 우선적으로 중복된 데이터가 있는지 확인한다.

#### (3) order_id 중 중복된 데이터가 있는지 확인
- value_counts(): 각 값이 전체에서 중복된 횟수를 리턴 (unique할 경우, 1을 리턴)
- max(): 최대값 가져오기
- value_counts().max(): 최대 중복된 데이터의 횟수 리턴

In [88]:
# 출력값이 1 이므로,  orders 데이터프레임의 order_id는 중복값이 없다.
orders['order_id'].value_counts().max()

1

In [89]:
# 출력값이 29 이므로,  payments 데이터프레임은 중복값을 가지고 있다.
payments['order_id'].value_counts().max()

29

In [90]:
payments['order_id'].value_counts()

fa65dad1b0e818e3ccc5cb0e39231352    29
ccf804e764ed5650cd8759557269dc13    26
285c2e15bebd4ac83635ccc563dc71f4    22
895ab968e7bb0d5659d16cd74cd1650c    21
fedcd9f7ccdc8cba3a18defedd1a5547    19
                                    ..
75d1b089800a77f3c4b653e297632423     1
1b41fbc113f284d979a4b5c94e2a918f     1
75b243673e2423675b74c7f34610610c     1
c33b68bf731561417deac5156e97409b     1
4396b37f55a457b870dcd9137cf07e84     1
Name: order_id, Length: 99440, dtype: int64

In [91]:
# 중복 확인...
payments[payments['order_id'] == 'fa65dad1b0e818e3ccc5cb0e39231352']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
4885,fa65dad1b0e818e3ccc5cb0e39231352,27,voucher,1,66.02
9985,fa65dad1b0e818e3ccc5cb0e39231352,4,voucher,1,29.16
14321,fa65dad1b0e818e3ccc5cb0e39231352,1,voucher,1,3.71
17274,fa65dad1b0e818e3ccc5cb0e39231352,9,voucher,1,1.08
19565,fa65dad1b0e818e3ccc5cb0e39231352,10,voucher,1,12.86
23074,fa65dad1b0e818e3ccc5cb0e39231352,2,voucher,1,8.51
24879,fa65dad1b0e818e3ccc5cb0e39231352,25,voucher,1,3.68
28330,fa65dad1b0e818e3ccc5cb0e39231352,5,voucher,1,0.66
29648,fa65dad1b0e818e3ccc5cb0e39231352,6,voucher,1,5.02
32519,fa65dad1b0e818e3ccc5cb0e39231352,11,voucher,1,4.03


#### (4) 중복된 order_id 에 대한 지불 가격을 합치기로 함
> 중복된 order_id 에 대해 orders 필드값이 덮어씌워져서 본래 orders 보다 많은 행이 생긴 것임

In [92]:
# groupby를 통해 해당 아이디가 사용한 총가격 계산
payments = payments.groupby('order_id').sum()
payments[payments.index == 'fa65dad1b0e818e3ccc5cb0e39231352']

,payment_sequential,payment_installments,payment_value
order_id,,,
fa65dad1b0e818e3ccc5cb0e39231352,435,29,457.99


In [93]:
payments.head()

,payment_sequential,payment_installments,payment_value
order_id,,,
00010242fe8c5a6d1ba2dd792cb16214,1,2,72.19
00018f77f2f0320c557190d7a144bdd3,1,3,259.83
000229ec398224ef6ca0657da4fc703e,1,5,216.87
00024acbcdf0a6daa1e931b038114c75,1,2,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,1,3,218.04


#### (5) orders 의 구매 날짜와 payments 의 총 지불 금액을 합침

In [94]:
# 구매날짜와 총지불금액이 모두 필요하므로,
# 둘중하나가 없는 데이터는 필요없다.
# 따라서, inner join을 채택
merged_order = pd.merge(orders, payments, on = 'order_id')
merged_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96460 entries, 0 to 96459
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       96460 non-null  object 
 1   customer_id                    96460 non-null  object 
 2   order_status                   96460 non-null  object 
 3   order_purchase_timestamp       96460 non-null  object 
 4   order_approved_at              96460 non-null  object 
 5   order_delivered_carrier_date   96460 non-null  object 
 6   order_delivered_customer_date  96460 non-null  object 
 7   order_estimated_delivery_date  96460 non-null  object 
 8   payment_sequential             96460 non-null  int64  
 9   payment_installments           96460 non-null  int64  
 10  payment_value                  96460 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 8.8+ MB


In [95]:
merged_order

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,6,3,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,1,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,3,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,1,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,1,28.62
...,...,...,...,...,...,...,...,...,...,...,...
96455,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,3,85.08
96456,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1,3,195.00
96457,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,1,5,271.01
96458,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,1,4,441.16


### [ (중요) pandas 로 날짜 다루기 ]

#### 시계열 자료와 pandas
- 년도별, 월별, 일별, 시별, 분별, 초별 등 시간의 흐름에 따라 관측된 자료
- pandas에서 시계열 자료를 손쉽게 다룰 수 있도록, <strong>datetime(datetime64)</strong> 자료형 제공
    - <strong>pandas.to_datetime()</strong>함수를 사용해서, 날짜와 시간을 나타내는 문자열을 datetime(datetime64) 자료형으로 변경 가능

#### (중요) pandas.to_datetime() 사용법
- 문자열 타입의 시간을 pandas의 datetime(datetime64) 형으로 변경 
- 주요 사용법
    - <strong> Series 변수 = to_datetime(Series 변수)</strong>
            - return된 Series 변수 데이터는 datetime64 형으로 변형되어 저장
    - <strong> Series 변수 = to_datetime(Series 변수, format = '어쩌구')</strong>
            - Series에 변환될 문자열이 특별한 포맷을 가져서, 자동변환이 어려울 경우
    - <strong> Series 변수 = to_datetime(Series 변수, error = 'raise' )</strong>
            - 수만가지 date자료중 포맷/형식이 일치하지 않아 에러가 발생했을 떄 처리 방법
            - 디폴트 값은 raise로 설정되어있다.
            - error 가능한 값 : ignore(무시), raise(에러발생), coerce(NaT로 값 변경해서 저장)
    
- 포맷 옵션 종류


| 표시 | 의미                                |
|------|:-------------------------------------|
| %y   | 연도를 축약해서 표시, 예: 21        |
| %Y   | 연도를 축약하지 않고 표시, 예: 2021 |
| %m   | 월을 두자릿 수로 표시, 예: 01 ~ 12  |
| %d   | 일을 두자릿 수로 표시, 예: 01 ~ 31  |
| %H   | 시간 표시(24시간), 예: 00 ~ 23        |
| %M   | 분 표시, 예: 00 ~ 59  |
| %S   | 초 표시, 예: 00 ~ 59        |

In [96]:
import pandas as pd
df = pd.DataFrame({'order':['2020-01-01 07:10:00',
                            '2020-01-08 07:20:30',
                            '2020-02-20 11:20:00',
                            '2020-02-20 04:40:50',
                            '2020-02-28 12:10:20',
                            '2019-01-10 17:23:50',
                            '2019-06-20 22:27:50',
                            '2019-06-20 21:15:59',
                            '2019-12-10 21:15:59',
                           ]})

In [97]:
# to_datetime 적용전 type은 object임을 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   order   9 non-null      object
dtypes: object(1)
memory usage: 200.0+ bytes


In [98]:
df['order'] = pd.to_datetime(df['order'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')

# to_datetime 적용 후 type이 datetime64로 변경되었음을 확인!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   order   9 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 200.0 bytes


#### (6) order 한 시간 정보 데이터만 datetime64 로 변환하기

In [99]:
merged_order

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,6,3,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,1,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,3,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,1,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,1,28.62
...,...,...,...,...,...,...,...,...,...,...,...
96455,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,1,3,85.08
96456,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1,3,195.00
96457,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,1,5,271.01
96458,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,1,4,441.16


In [100]:
# order_purchase_timestamp가 현재 object 타입으로 되어있는 것을 확인
merged_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96460 entries, 0 to 96459
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       96460 non-null  object 
 1   customer_id                    96460 non-null  object 
 2   order_status                   96460 non-null  object 
 3   order_purchase_timestamp       96460 non-null  object 
 4   order_approved_at              96460 non-null  object 
 5   order_delivered_carrier_date   96460 non-null  object 
 6   order_delivered_customer_date  96460 non-null  object 
 7   order_estimated_delivery_date  96460 non-null  object 
 8   payment_sequential             96460 non-null  int64  
 9   payment_installments           96460 non-null  int64  
 10  payment_value                  96460 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 8.8+ MB


In [101]:
merged_order['order_purchase_timestamp '] = pd.to_datetime(merged_order['order_purchase_timestamp'],
                                                                        format = '%Y-%m-%d %H:%M:%S',
                                                                        errors = 'raise')

# order_purchase_timestamp 의 타입이 datetime64로 변경
merged_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96460 entries, 0 to 96459
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96460 non-null  object        
 1   customer_id                    96460 non-null  object        
 2   order_status                   96460 non-null  object        
 3   order_purchase_timestamp       96460 non-null  object        
 4   order_approved_at              96460 non-null  object        
 5   order_delivered_carrier_date   96460 non-null  object        
 6   order_delivered_customer_date  96460 non-null  object        
 7   order_estimated_delivery_date  96460 non-null  object        
 8   payment_sequential             96460 non-null  int64         
 9   payment_installments           96460 non-null  int64         
 10  payment_value                  96460 non-null  float64       
 11  order_purchase_

#### (7) pandas.DataFrame.copy
- 기존 데이터프레임의 수정을 방지
- 데이터프레임 중 일부를 선택 후, 조작하면 해당 데이터프레임도 변경
- copy() 를 통해, 복사본을 만들어서 조작하여, 원본 데이터프레임은 보존 가능

In [102]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()
merged_order_payment_date.head()

,order_purchase_timestamp,payment_value
0,2017-10-02 10:56:33,38.71
1,2018-07-24 20:41:37,141.46
2,2018-08-08 08:38:49,179.12
3,2017-11-18 19:28:06,72.20
4,2018-02-13 21:18:39,28.62


In [103]:
# 아직 이유를 찾지 못했지만 현재 order_purchase_timestamp가 이 단계에서 object 타입으로 나타남
# 따라서 형변환을 시켜줘야한다.
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'],
                                                                        format = '%Y-%m-%d %H:%M:%S',
                                                                          errors = 'raise')

In [104]:
merged_order_payment_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96460 entries, 0 to 96459
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_purchase_timestamp  96460 non-null  datetime64[ns]
 1   payment_value             96460 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.2 MB


### [ (중요) pandas.Grouper ]
- pandas groupby 명령에 보다 세부적인 grouping 이 가능토록 하는 명령
- pandas groupby 함수와 함께 쓰여서, 시간별로 데이터를 분류할 수 있는 기능
- 특정 시간별로 grouping 할 수 있음

```
데이터프레임.groupby(pd.Groper(key='그루핑기준이되는 컬럼', freq='세부 기준'))

```

> freq 옵션: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

#### (8) 시간대별 거래액 확인하기 

In [108]:
# Groupby를 이용해서 key인 order_purchase_timestamp를 월별단위로 (sum)묶었다.
merged_order_month_sum = merged_order_payment_date.groupby(pd.Grouper(key='order_purchase_timestamp', freq='M')).sum() # key 는 기본이 index 임
merged_order_month_sum.head()

,payment_value
order_purchase_timestamp,
2016-10-31,47271.20
2016-11-30,0.00
2016-12-31,19.62
2017-01-31,127430.74
2017-02-28,269458.98


#### (9) 시각화해서 트렌드 확인하기

In [109]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

# 직장이나 업무에서 쓰기에는 다른사람들 눈에는 확연히 들어오지 않기에
# 세밀화 과정을 거쳐야한다.
merged_order_month_sum.iplot(kind = 'bar', theme = 'white')

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
    <font size="3em" style="font-weight:bold;color:#3f8dbf;">월별 평균 거래액</font>
</div>

In [110]:
# mean함수를 써서 모든 거래액의 평균을 산출한다.
merged_order_month_sum['payment_value'].mean()

670420.9934782615

In [112]:
# (참고) 파이썬의 슬라이스를 사용함으로써 월구간 사이의 평균값을 산출해낼 수 있다.
merged_order_month_sum['payment_value'][3:5].mean()

198444.85999999975

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
    <font size="3em" style="font-weight:bold;color:#3f8dbf;">최대 거래액을 기록한 월</font>
</div>

In [114]:
# 시리즈의 값으로 출력
merged_order_month_sum['payment_value'].max()

1153393.2200000011

In [116]:
# 데이터프레임의 형태로서 출력
merged_order_month_sum[merged_order_month_sum['payment_value'] == merged_order_month_sum['payment_value'].max()]

,payment_value
order_purchase_timestamp,
2017-11-30,1153393.22
